In [2]:
from qiskit_ibm_runtime.fake_provider import FakeKyiv, FakeKyoto
from qiskit import QuantumCircuit
from qiskit.circuit.library import QFT
from qiskit.circuit import Instruction, Parameter

from colorama import Fore


from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_aer import AerSimulator
from qiskit.compiler import transpile
from qiskit.circuit.library import QFT
from qiskit_ibm_runtime.fake_provider import FakeMelbourne

from utils import connections_backend, connections_circuit



In [3]:
backend_kyiv = FakeKyiv()
backend_kyoto= FakeKyoto()

In [27]:
qc = QFT(4)
pm_kyiv = generate_preset_pass_manager(optimization_level=1, backend=backend_kyiv)
circuit_isa_kyiv = pm_kyiv.run(qc)

pm_kyoto = generate_preset_pass_manager(optimization_level=1, backend=backend_kyoto)
circuit_isa_kyoto = pm_kyoto.run(qc)



In [4]:
grader_operation_check_message : str = 'Basis gate check passed!'
def grader_operation_check(backend : any, circuit : QuantumCircuit):

    """
    Takes a backend and a circuit and see whether the gates in the circuit are supported by the backend

    Parameters : 
                backend : Target backend
                circuit : Quantum Circuit whose gates are to be checked with the gates in the target backend
    """

    op_names_backend : list[str] = backend.operation_names
    op_names_circuit_dict : dict[str,int]  = circuit.count_ops().items()

    for keys, val in op_names_circuit_dict:
        if keys in op_names_backend:
            None
        else : 
            print(Fore.RED + f'The backend does not support {keys} gate')

    return print(Fore.GREEN + grader_operation_check_message)

In [5]:
grader_operation_check(backend_kyiv, circuit_isa_kyiv)


Basis gate check passed!


In [6]:
grader_operation_check(backend_kyiv, circuit_isa_kyoto)


Basis gate check passed!


In [7]:
grader_connection_check_correct_message = 'The connections match with the backend'
grader_connection_check_incorrect_message = 'The connections does not match with the backend'

def grader_connection_check(backend: any, circuit : QuantumCircuit)-> None:

    """"
    Takes in a Quantum Circuit and a Backend and checks whether the conections in the circuit matches 
    with the topology of the backend. 

    Parameters : 
                backend : Target backend in which the circuit must fit.
                circuit : Circuit whose connections are to be matched with the backend

    Returns : Message : Whether the circuit fits in the topology of the backend. 
    """
    conn_backend = connections_backend(backend)
    conn_circ = connections_circuit(circuit)

    gate_conn_set_backend : dict[str,set] = {}
    for keys,items in conn_backend.items():
        gate_conn_set_backend[keys] = set(items)

    gate_conn_set_circuit : dict[str,set] = {}
    for keys,items in conn_circ.items():
        gate_conn_set_circuit[keys] = set(items)

    check_connections : list[int] = []
    for keys, items in gate_conn_set_circuit.items():
        check_connections.append(int(gate_conn_set_circuit[keys].issubset(gate_conn_set_backend[keys])))

    if 0 in check_connections:
        print(Fore.RED + grader_connection_check_incorrect_message)

    else : 
        print(Fore.GREEN +grader_connection_check_correct_message )

In [8]:
grader_connection_check(backend_kyiv, circuit_isa_kyiv)


The connections match with the backend


In [9]:
grader_connection_check(backend_kyiv, circuit_isa_kyoto)


The connections does not match with the backend


In [ ]:
circuit_target = QFT(4)
simulator = AerSimulator(method = "statevector")
backend_melbourne = FakeMelbourne()
circuit_target = transpile(circuit_target, backend = simulator)
circuit_test = transpile(circuit_target, backend = backend_melbourne)

In [136]:
def counts(circuit : QuantumCircuit, backend = simulator) -> dict[str, int]:
    """
    Function to add measurements to a provided circuit and run it on the Aer Simulator.

    Parameters 
        circuit : Quantum Circuit 
        backend : default = AerSimulator

    Returns  
        counts in a dict.

    """

    if 'measure' in circuit.count_ops():
        None
    else : 
        circuit.measure_active()
    result = simulator.run(circuit, backend = simulator).result()
    counts = result.get_counts()

    return counts

In [137]:
def grader_circuit_accuracy(circuit_target: QuantumCircuit, circuit_test : QuantumCircuit, backend : any = simulator):
    """
    Takes the test circuit and compares it with the target circuit
    
    """

    circuit_target = transpile(circuit_target, backend = simulator)
    circuit_test = transpile(circuit_test, backend = simulator)

    counts_target = counts(circuit_target)
    counts_test = counts(circuit_test)
    
    return counts_target, counts_test


In [138]:
grader_circuit_accuracy(circuit_target, circuit_test)

({'1001': 69,
  '0001': 75,
  '1010': 58,
  '0111': 57,
  '1011': 63,
  '1000': 73,
  '0100': 50,
  '1111': 58,
  '0011': 72,
  '0101': 78,
  '1101': 64,
  '0000': 62,
  '1100': 61,
  '0010': 55,
  '1110': 66,
  '0110': 63},
 {'0001': 60,
  '1001': 65,
  '1010': 90,
  '0100': 61,
  '1100': 64,
  '1011': 58,
  '0000': 74,
  '0101': 67,
  '1101': 66,
  '1111': 63,
  '1110': 76,
  '1000': 67,
  '0111': 59,
  '0110': 55,
  '0010': 37,
  '0011': 62})

{'0011': 66, '1100': 73, '0001': 75, '1101': 73, '1111': 70, '1011': 58, '0010': 69, '1110': 43, '1000': 65, '1010': 62, '1001': 61, '0100': 65, '0110': 62, '0101': 61, '0000': 65, '0111': 56}
{'1000': 61, '1110': 59, '1111': 74, '0110': 67, '0100': 62, '1010': 55, '0011': 63, '0010': 58, '1101': 52, '1001': 71, '0111': 61, '0000': 79, '0101': 59, '0001': 78, '1100': 66, '1011': 59}


In [119]:
counts(circuit_test)

{'1101': 65,
 '0110': 55,
 '1111': 72,
 '1110': 53,
 '0001': 49,
 '1010': 67,
 '1100': 57,
 '1000': 78,
 '0010': 66,
 '0000': 76,
 '1011': 50,
 '0101': 76,
 '0111': 66,
 '0100': 67,
 '0011': 63,
 '1001': 64}